### titanic LLM

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install -r requirements.txt

In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
from sklearn.model_selection import train_test_split
import torch

In [2]:
df = pd.read_csv('titanic.csv')

In [3]:
# Wypełnienie brakujących wartości
df['Age'].fillna(df['Age'].median(), inplace=True)
df['Embarked'].fillna('S', inplace=True)

# Przekształcenie cech na tekst
def row_to_text(row):
    return (
        f"Passenger {row['Name']} is a {row['Sex']} aged {int(row['Age'])} "
        f"traveling in class {row['Pclass']} who embarked from {row['Embarked']}."
    )

df['text'] = df.apply(row_to_text, axis=1)

# Label (etykieta) - czy pasażer przeżył
df['label'] = df['Survived']


In [4]:
train_df, test_df = train_test_split(df[['text', 'label']], test_size=0.2, random_state=42)

In [5]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [6]:
test_dataset

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 179
})

In [7]:
model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

/Users/maciektaraday/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/712 [00:00<?, ? examples/s]

Map:   0%|          | 0/179 [00:00<?, ? examples/s]

In [9]:
train_dataset = train_dataset.remove_columns(['text', '__index_level_0__'])
test_dataset = test_dataset.remove_columns(['text', '__index_level_0__'])

train_dataset = train_dataset.rename_column('label', 'labels')
test_dataset = test_dataset.rename_column('label', 'labels')

train_dataset.set_format('torch')
test_dataset.set_format('torch')

In [11]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
import accelerate
print(accelerate.__version__)

1.0.1


In [13]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
)

In [14]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)


In [16]:
trainer.train()

/Users/maciektaraday/venv/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.530500,0.500480,0.804469,0.768212,0.753247,0.783784
2,0.584400,0.482157,0.815642,0.784314,0.759494,0.810811
3,0.482000,0.483942,0.821229,0.786667,0.776316,0.797297


TrainOutput(global_step=135, training_loss=0.48752246874350086, metrics={'train_runtime': 7629.4213, 'train_samples_per_second': 0.28, 'train_steps_per_second': 0.018, 'total_flos': 282950363529216.0, 'train_loss': 0.48752246874350086, 'epoch': 3.0})

In [17]:
trainer.evaluate()

{'eval_loss': 0.483941912651062,
 'eval_accuracy': 0.8212290502793296,
 'eval_f1': 0.7866666666666666,
 'eval_precision': 0.7763157894736842,
 'eval_recall': 0.7972972972972973,
 'eval_runtime': 193.3252,
 'eval_samples_per_second': 0.926,
 'eval_steps_per_second': 0.016,
 'epoch': 3.0}

In [18]:
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted_class = torch.argmax(probabilities).item()
    return predicted_class, probabilities.detach().numpy()

# Przykład
new_passenger = "Passenger John Doe is a male aged 28 traveling in class 3 who embarked from S."
predicted_class, probabilities = predict(new_passenger)
print(f"Predykcja: {'Przeżył' if predicted_class == 1 else 'Nie przeżył'}")
print(f"Prawdopodobieństwa: {probabilities}")


Predykcja: Nie przeżył
Prawdopodobieństwa: [[0.89165974 0.10834032]]
